In [ ]:
%run 03_neural_network_attempt.ipynb
import numpy as np
import pandas as pd
import importlib
import config
from datetime import datetime, timedelta
importlib.reload(config)

import warnings
# Suppress all warnings (not recommended unless you're sure)
warnings.filterwarnings("ignore")

In [2]:
##OPTIMISING

#learning rate
#min weight
#algo
#epochs
#window size

# depending on computational power, this might take a bit...

In [3]:
algos = [config.MAX_PROFIT, config.RISK_PARITY, config.MAX_SHARPE]
epochs = ["14D","20D"]

max_return = 0
for window_size in range(50,100,10):
    for learning_rate in np.arange(0.01, 0.02, 0.01):
        for min_weight in np.arange(0.02, 0.03, 0.01):
            for algo in algos:
                for epoch in epochs:
                    print("now optimising:", window_size, " " , learning_rate," ", min_weight," ", algo, " ", epoch)
                    #make predictions for long timeframe and copy the mse in new column

                    #generating training and test data:
                    #get_training_and_test_data(start_date, length, portfolio = config.MAX_SHARPE, window_size = 30, min_weight = 0.1,rf_rate = 0.02):
                    train_set, test_set = get_training_and_test_data("2012-01-01",500, portfolio = algo, window_size = 30, min_weight = 0.01,rf_rate = 0.02, freq = "D")

                    #train_set.iloc[:,0:11]     this is used as an input to train the model (only first 11 columns are the input indicators)
                    #train_set.iloc[:,11:]      this is used as targets to train the model (last 4 columns are the optimal weights )

                    #train neural network

                    trained_weights_input_hidden, trained_weights_hidden_output = train_neural_network(train_set.iloc[:,0:11], train_set.iloc[:,11:], learning_rate=0.1)

                    #make predictions
                    test_set, weight_predictions = predict_weights(test_set,trained_weights_input_hidden, trained_weights_hidden_output, freq = "D")

                    date_range = pd.date_range(start=weight_predictions.index.min(), end=weight_predictions.index.max(), freq='D')
                    start_date = date_range[0] - timedelta(days=1)
                    df_prices = get_indices(start_date.strftime("%Y-%m-%d"), date_range[-1].strftime("%Y-%m-%d")).pct_change().dropna()

                    #calculate and plot returns
                    preturn = (weight_predictions * df_prices).sum(axis=1)
                    total_return = preturn.cumsum()
                    if total_return[-1] > max_return:
                        max_return = total_return[-1]
                        optimal_combination = (window_size, " ", learning_rate," ", min_weight," ", algo," ",  epoch)
                        print("max_return: ", max_return)
                        print("window_size: ", window_size)
                        print("learning_rate: ", learning_rate)
                        print("min_weight: ", min_weight)
                        print("algo: ", algo)
                        print("epoch: ", epoch)

                                    

now optimising: 50   0.01   0.02   Max Profit   14D
Day 0, Mean Squared Error: 0.29522597453166527
Day 50, Mean Squared Error: 0.13169374826587446
Day 100, Mean Squared Error: 0.12576257314357936
Day 150, Mean Squared Error: 0.1274426566573717
Day 200, Mean Squared Error: 0.0685401321494972
Day 250, Mean Squared Error: 0.15735678137517312
Day 300, Mean Squared Error: 0.19583305709796198
Day 350, Mean Squared Error: 0.15176519285868353
Day 400, Mean Squared Error: 0.11118784699446556
Day 450, Mean Squared Error: 0.04223313223742387
Day 500, Mean Squared Error: 0.12202978764575026
max_return:  0.09857675589416946
window_size:  50
learning_rate:  0.01
min_weight:  0.02
algo:  Max Profit
epoch:  14D
now optimising: 50   0.01   0.02   Max Profit   20D
Day 0, Mean Squared Error: 0.29522597453166527
Day 50, Mean Squared Error: 0.13169374826587446
Day 100, Mean Squared Error: 0.12576257314357936
Day 150, Mean Squared Error: 0.1274426566573717
Day 200, Mean Squared Error: 0.0685401321494972
Day

In [4]:
optimal_combination

(50,
 ' ',
 np.float64(0.01),
 ' ',
 np.float64(0.02),
 ' ',
 'Max Profit',
 ' ',
 '14D')